Recalculate positions

In [1]:
%gui qt
# Generate the renaming first
import converter

converter.main()  # Run it twice, once for prescan, once for hr

SystemExit: 0

C:\Users\lleti\AppData\Local\conda\conda\envs\golgiqc\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
# Some utilities
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import glob
import re,json
import shutil


def findTiffs(foldname):
    flist = []
    tiff_list = []
    dirs = []
    dirs.append(foldname)
    while dirs:
        fname = dirs.pop()
        fname_tif = str(fname) + '\\*.tif*'
        flist = glob.glob(fname_tif)
        if not flist:
            newdirs = ([f for f in glob.glob(fname + '\\*') if os.path.isdir(f)])
            for el in newdirs:
                dirs.append(el)
        else:
            for el in flist:
                tiff_list.append(el)
    return tiff_list


def filterPick(myList, myString):
    pattern = re.compile(myString);
    indices = [i for i, x in enumerate(myList) if pattern.search(x)]
    return indices



In [2]:
main_folder = "."
cells_file = "\\new_selected_cells.csv"
# Load renamed directory
hres = main_folder + "\\renamed\\hr"
prescan = main_folder+"\\renamed\\prescan"
flist_hres = os.listdir(hres) # List of High resolution jobs
flist_prescan = os.listdir(prescan) # List of prescans

# Load new selected_cells.csv
file_cells = main_folder + cells_file
with open(file_cells, 'r') as myfile:
        data = myfile.read().replace('\"', '')
with open(file_cells, 'w') as myfile:
        myfile.write(data)

table  = pd.read_csv(file_cells,',')
list_copied = {}
new_hres = main_folder+"\\new_hr"
if not os.path.exists(new_hres):
    os.mkdir(new_hres)
# For each cell in the table
for i in range(len(table)):
    reg_exp = "X" + str(table.at[i,"Metadata_X"]).zfill(2)+"--"+ "Y" + str(table.at[i,"Metadata_Y"]).zfill(2)
    ind = filterPick(flist_prescan, reg_exp)
    if not ind:
        print("Prescan files not found.")
        continue
    # Get the prescan data
    prescan_folder_s = prescan + "\\"+flist_prescan[ind[0]]
    # Get the high res data
    ind = filterPick(flist_hres, reg_exp)
    name_or = "X" + str(table.at[i,"Metadata_X"]).zfill(2) + "--" + "Y" + str(table.at[i,"Metadata_Y"]).zfill(2)+"_"+str(i).zfill(4)
    name = str(i)+"_"+str(table.at[i,"Gene"]+"_"+name_or)
    if(len(ind)==0):           
        ind = filterPick(flist_hres, ".*--"+ "Y" + str(table.at[i,"Metadata_Y"]).zfill(2)+".*")
        if(len(ind)==0):
            print("HR files not found for "+name)
            continue      
    cell_dir = new_hres+"\\"+name
    if not os.path.exists(cell_dir):
            os.mkdir(cell_dir)
    alltifs = findTiffs(prescan_folder_s)
    for el in alltifs:
            shutil.copy2(el, cell_dir)
            # print("Copying "+el+" to :"+cell_dir)
        
    hres_dir = hres + "\\"+flist_hres[ind[0]]
        
    hres_s = os.listdir(hres_dir)
        # Filter only 10x--z1
    indtifs= filterPick(hres_s, '.*10x--z1.*.tif')
    hres_tifs =[hres_s[el] for el in indtifs]
    for el in hres_tifs:
            shutil.copy2(hres_dir + "\\"+el, cell_dir)
            # print("Copying "+el+" to :"+cell_dir)
    print("HR files COPIED for "+name)
# Find matching and extract context

HR files COPIED for 0_COPG1_X03--Y03_0000
HR files COPIED for 1_COPG1_X03--Y03_0001
HR files COPIED for 2_COPG1_X00--Y03_0002
HR files COPIED for 3_COPG1_X01--Y03_0003
HR files COPIED for 4_COPG1_X02--Y03_0004
HR files COPIED for 5_COPG1_X03--Y03_0005
HR files COPIED for 6_COPB2_X01--Y00_0006
HR files COPIED for 7_COPB2_X01--Y00_0007
HR files COPIED for 8_COPB2_X02--Y00_0008
HR files COPIED for 9_COPB2_X02--Y00_0009
HR files COPIED for 10_COPB1_X00--Y24_0010
HR files COPIED for 11_COPB1_X03--Y24_0011
HR files COPIED for 12_DNM1_X00--Y02_0012
HR files COPIED for 13_DNM1_X03--Y02_0013
HR files COPIED for 14_DNM1_X03--Y02_0014
HR files COPIED for 15_DNM1_X01--Y02_0015
HR files COPIED for 16_DNM1_X02--Y02_0016
HR files COPIED for 17_DNM1_X02--Y02_0017
HR files not found for 18_DNM1_X02--Y12_0018
HR files not found for 19_DNM1_X02--Y12_0019
HR files COPIED for 20_DENND4C_X01--Y30_0020
HR files COPIED for 21_DENND4C_X00--Y30_0021
HR files COPIED for 22_DENND4C_X02--Y30_0022
HR files COPIED f

### Extract organelle coordinates

In [3]:
def savePrescanShift(main_folder,saving_folder, separator = ",",organ_name="organelle"):
    file_cells = main_folder + "\\new_selected_cells.csv"
    table = pd.read_csv(file_cells,separator)
    # as a shorthand, extract some columns from the data frame and give
    # them a convenient name
    loc_center = pd.DataFrame(table, columns=["Location_Center_X", "Location_Center_Y"])
    loc_golgi_mean = pd.DataFrame(table, columns=["Mean_Golgi_AreaShape_Center_X", "Mean_Golgi_AreaShape_Center_Y"])
    samples=[]
    for i in range(loc_center.shape[0]):
        samples.append("X" + str(table.at[i,"Metadata_X"]).zfill(2) + "--" + "Y" + str(table.at[i,"Metadata_Y"]).zfill(2)+"_"+str(i).zfill(4))

    indices = pd.Index(samples, dtype='object')
    loc_center = loc_center.set_index(indices)
    loc_golgi_mean  =loc_golgi_mean.set_index(indices)

    loc_center.to_json(saving_folder + '\\new_center.json')
    loc_golgi_mean.to_json(saving_folder+'\\new_center_'+organ_name+'.json')
    print("File new_center.json and new_center_"+organ_name+".json generated successfully.") 

In [4]:
savePrescanShift(main_folder,main_folder+"\\new_hr")

File new_center.json and new_center_organelle.json generated successfully.
